In [1]:
import tensorflow as tf
import numpy as np
import os
import string
import gensim
import fasttext

In [76]:
del_letters = string.punctuation
del_tran_table = str.maketrans(del_letters, " " * len(del_letters))

relations = ['Cause-Effect', 'Component-Whole', 'Entity-Destination', 'Product-Producer', 'Entity-Origin',
             'Member-Collection', 'Message-Topic', 'Content-Container', 'Instrument-Agency', 'Other']


def preprocess(sentence):
    sentence = sentence.translate(del_tran_table)
    tokens = nltk.tokenize.word_tokenize(sentence)
    return tokens


dataset_dir = os.path.join("..", "dataset", "")
train_path = os.path.join(dataset_dir, "train.txt")

# sentences = list()
texts = list()
labels = list()
with open(train_path) as fp:
    while True:
        line = fp.readline()
        label = fp.readline()
        if not line or not label:
            break
        texts.append(line.split(" ", 1)[1])
#         sentence = preprocess(line.split(" ", 1)[1])
#         sentences.append(sentence)
#         if len(sentence) > max_len:
#             max_len = len(sentence)
        labels.append(label.split('(')[0])
num_sample = len(labels)

6400

In [82]:
max_words = 10000
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(texts)
sentences = tokenizer.texts_to_sequences(texts)
word_index = tokenizer.word_index

maxlen = max(map(len, sentences))

data = tf.keras.preprocessing.sequence.pad_sequences(sentences, maxlen=maxlen)
data

array([[   0,    0,    0, ...,    2, 1875, 1876],
       [   0,    0,    0, ...,    2,    3, 1877],
       [   0,    0,    0, ..., 1654,  849,  850],
       ...,
       [   0,    0,    0, ...,    5,  395,  405],
       [   0,    0,    0, ...,    1,  779,  276],
       [   0,    0,    0, ...,  536,   15, 3990]], dtype=int32)

In [25]:
# model = gensim.models.Word2Vec(sentences, min_count=1)
model = fasttext.load_model(os.path.join('..', 'res', 'dbpedia.ftz'))
vec_len = model.get_dimension()


def get_vec(word):
    return model[word]

In [29]:
X = list()
seq_lens = list()
seq_masks = list()
for sentence in sentences:
    seq = list()
    sentence_len = len(sentence)
    seq_lens.append(sentence_len)
    seq_masks.append(sentence_len * [True] + (max_len-sentence_len) * [False])
    for word in sentence:
        word_vec = get_vec(word)
        seq.append(word_vec)
    for i in range(max_len-sentence_len):
        seq.append([0] * vec_len)
    X.append(seq)
X = np.asarray(X)
seq_lens = np.asarray(seq_lens)
seq_masks = np.asarray(seq_masks)


Y = list(map(relations.index, labels))
Y = np.eye(len(relations))[Y]

In [27]:
train_ratio = 0.75
num_train = int(num_sample * train_ratio)
rand_indices = np.random.permutation(np.arange(num_sample))
indices_train = rand_indices[:num_train]
indices_test = rand_indices[num_train:]

In [32]:
X_train = X[indices_train]
X_test = X[indices_test]
Y_train = Y[indices_train]
Y_test = Y[indices_test]
# seq_lens_train = seq_lens[indices_train]
# seq_lens_test = seq_lens[indices_test]
seq_masks_train = seq_masks[indices_train]
seq_masks_test = seq_masks[indices_test]

In [57]:
inputs = tf.keras.Input(shape=(max_len, vec_len))
x = tf.keras.layers.LSTM(100, dropout=0.2, recurrent_dropout=0.2)(
    inputs=inputs,)
outputs = tf.keras.layers.Dense(len(relations), activation='softmax')(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs)

model.compile(loss='categorical_crossentropy',
              optimizer='adam', metrics=['accuracy'])

model.fit(X_train, Y_train, epochs=1)

150/150 [==============================] - 13s 82ms/step - loss: 2.2813 - accuracy: 0.1464
